# Amazon SageMaker PyTorch コンテナを用いた MNIST の学習と推論 

## 目次

1. [背景](#1.背景)
1. [セットアップ](#2.セットアップ)
1. [データ](#3.データ)
1. [学習](#4.学習)
1. [ハイパーパラメータ調整を用いた学習](#5.ハイパーパラメータ調整を用いた学習)
1. [ホスティング](#6.ホスティング)
1. [リソースの削除](#7.リソースの削除)

---

## 1.背景

MNISTは、手書き文字の分類に広く使用されているデータセットです。 70,000個のラベル付きの28x28ピクセルの手書き数字のグレースケール画像で構成されています。 データセットは、60,000個のトレーニング画像と10,000個のテスト画像に分割されます。 手書きの数字 0から9の合計10のクラスがあります。 このチュートリアルでは、PyTorch を使用して SageMaker で MNIST モデルをトレーニングおよびテストする方法を示します。 また、SageMaker の自動モデルチューニングを使用して適切なハイパーパラメーターを選択し、最適なモデルを取得する方法をご説明します。

SageMaker の PyTorch の詳細については、[sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) と [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) のレポジトリをご参照ください。

---

## 2.セットアップ

SageMaker セッションを作成し、設定を開始します。

- 学習およびモデルデータに使用する S3 バケットとプレフィックスは、ノートブックインスタンス、トレーニング、およびホスティングと同じリージョン内にある必要があります。
- データへの学習およびホスティングアクセスを提供するために使用される IAM ロール arn を用います。 ノートブックインスタンス、学習インスタンス、および/またはホスティングインスタンスに複数のロールが必要な場合は、 `sagemaker.get_execution_role（）` を、適切な IAM ロール arn 文字列に置き換えてください。


In [ ]:
import sagemaker
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

このノートブックのコードは、以前からのノートブックインスタンスで実行する場合と、SageMaker Studio のノートブックで実行する場合とで挙動が異なります。以下のセルを実行することで、いまの実行環境が以前からのノートブックインスタンスなのか、SageMaker Studio のノートブックなのかを判定して、`on_studio`に記録します。この結果に基づいて、以降のノートブックの実行を次のように変更します。

- データセットの展開先を変更します。SageMaker Studio を利用する場合、home のディレクトリは EFS をマウントして実現されており、データセットを展開する際にやや時間を要します。そこで home 以外のところへ展開するようにします。
- SageMaker Studio では学習・推論の local モードがサポートされていません。従って、Studio では local モードを行わないようにします。

In [ ]:
import os, json
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get("DomainId")
        on_studio = True if domain_id is not None else False
print("Is this notebook runnning on Studio?: {}".format(on_studio))

## 3.データ
### 3.1.データの取得

In [ ]:
!aws s3 cp s3://fast-ai-imageclas/mnist_png.tgz . --no-sign-request
if on_studio:
    !tar -xzf mnist_png.tgz -C /opt/ml --no-same-owner
else:
    !tar -xvzf  mnist_png.tgz

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os

root_dir_studio = '/opt/ml'
data_dir = os.path.join(root_dir_studio,'data') if on_studio else 'data'
training_dir = os.path.join(root_dir_studio,'mnist_png/training') if on_studio else 'mnist_png/training'
test_dir = os.path.join(root_dir_studio,'mnist_png/testing') if on_studio else 'mnist_png/testing'

os.makedirs(data_dir, exist_ok=True)

training_data = datasets.ImageFolder(root=training_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))
test_data = datasets.ImageFolder(root=test_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))

training_data_loader = DataLoader(training_data, batch_size=len(training_data))
training_data_loaded = next(iter(training_data_loader))
torch.save(training_data_loaded, os.path.join(data_dir, 'training.pt'))

test_data_loader = DataLoader(test_data, batch_size=len(test_data))
test_data_loaded = next(iter(test_data_loader))
torch.save(test_data_loaded, os.path.join(data_dir, 'test.pt'))

### 3.2.データをS3にアップロードする
データセットを S3 にアップロードするには、 `sagemaker.Session.upload_data` 関数を使用します。 戻り値として入力した S3 のロケーションは、後で学習ジョブを実行するときに使用します。

In [ ]:
inputs = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## 4.学習
### 4.1.学習スクリプト
`mnist.py` スクリプトは、SageMaker モデルの学習とホスティングに必要なすべてのコードを提供します（`model_fn` はモデルをロードするための関数です）。
学習スクリプトは、SageMaker の外部で実行するトレーニングスクリプトに非常に似ていますが、次のようなさまざまな環境変数を通じてトレーニング環境に関する有用なプロパティにアクセスできます。

* `SM_MODEL_DIR`： モデルアーティファクトを書き込むディレクトリへのパスを表す文字列。モデルは、推論用ホスティングのためにS3にアップロードされます。
* `SM_NUM_GPUS`： 現在のコンテナで利用可能な GPU の数。
* `SM_CURRENT_HOST`： コンテナネットワーク上の現在のコンテナの名前。
* `SM_HOSTS`： すべてのホストを含む JSON エンコードリスト。

`fit（）`メソッドの呼び出しで1つの入力チャンネル `training` が使用されたとすると、 `SM_CHANNEL_ [channel_name]`の形式に従って以下が設定されます：

* `SM_CHANNEL_TRAINING`：`training` チャンネルのデータを含むディレクトリへのパスを表す文字列。

学習に関する環境変数の詳細については、[SageMaker Containers](https://github.com/aws/sagemaker-containers) をご覧ください。

典型的なトレーニングスクリプトの書き方は下記の通りです。入力チャンネルからデータをロードし、ハイパーパラメーターで学習の設定、モデルを学習、モデルを `model_dir` に保存し、そこから学習済みモデルのホスティングを行います。 ハイパーパラメーターは引数としてスクリプトに渡され、 `argparse.ArgumentParser` インスタンスとして取得できます。

下記は、このノートブックで使われるスクリプト ` mnist.py` です。SageMaker PyTorch Container で定義されるデフォルトの実装方法に従い [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) 、 `input_fn` 、` predict_fn` 、 `output_fn` および ` transform_fn` の実装を使用します。メインガード  (``if __name__=='__main__':``)  の中のスクリプトは、学習時にのみ実行され、モデルのホスティング時には実行されません。上記のように、必要な `mnist.py` スクリプトに ` model_fn` を実装しています。

In [ ]:
!pygmentize mnist.py

### 4.2.Estimatorの定義
学習の条件を設定するため、Estimator クラスの子クラスの PyTorch オブジェクトを作成します。 ここでは、PyTorchスクリプト、IAMロール、および（ジョブごとの）ハードウェア構成を渡す PyTorch Estimator を定義しています。また合わせてローカルの `source_dir` を指定することで、依存するスクリプト群をコンテナにコピーして、学習時に使用することが可能です。

`mnist.py` で定義されているハイパーパラメータをレンジの形で指定してハイパーパラメータ探索を行う場合は、[ハイパーパラメーター調整ジョブをセットアップする](### ハイパーパラメーター調整ジョブをセットアップする) で紹介しています。

In [ ]:
from sagemaker.pytorch import PyTorch

if on_studio:
    instance_type = 'ml.m4.xlarge'
else:
    instance_type = 'local'

estimator = PyTorch(entry_point="mnist.py",
                    role=role,
                    framework_version='1.8.1',
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 1,
                        'backend': 'gloo'
                    })

### 4.3.ローカルモードでの学習の実行

***以前のノートブックインスタンスのみローカルモードを利用できます。Studio の場合は ml.m4.xlarge を利用します。***


`fit()` メソッドで学習ジョブを実行します。`entry_point` で指定したローカルのスクリプトが、学習用のコンテナ内で実行されます。

ノートブックインスタンスのCPUで学習する場合はinstance_type = 'local'、GPUで学習する場合はlocal_gpuを指定します。インスタンス数は、ノートブックインスタンスの数、すなわち1になるため、 train_instance_countに指定された値が1より大きい場合も1として扱われますが、traini_instance_count > 1 の分散学習はローカル環境でも擬似的に検証できます。

In [ ]:
estimator.fit({'training': inputs})

### 4.4.ローカルモードでモデルの検証

***以前のノートブックインスタンスのみローカルモードを利用できます。Studio の場合は ml.m4.xlarge を利用します。***

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

### 4.5.画像データに対する予測の実行

正規化していない MNIST データを正解データとして確認するために読み込んでおきます。テストデータから5枚をランダムにサンプリングして予測してみます。予測のために`predictor`に画像を入力します。

In [ ]:
%matplotlib inline
import random
import numpy as np
import matplotlib.pyplot as plt

raw_test_data = datasets.ImageFolder(root=test_dir,
                                        transform=transforms.Compose([
                                        transforms.Grayscale(),
                                        transforms.ToTensor()]))
num_samples = 5
indices = random.sample(range(len(raw_test_data) - 1), num_samples)
raw_images = np.array([raw_test_data[i][0].numpy() for i in indices])
raw_labels = np.array([raw_test_data[i][1] for i in indices])
images = np.array([test_data[i][0].numpy() for i in indices])

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(raw_images[i].reshape(28, 28), cmap='gray')
    plt.title(raw_labels[i])
    plt.axis('off')

prediction = predictor.predict(images)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))

## 5 ハイパーパラメータ調整を用いた学習
### 5.1.ハイパーパラメーター調整ジョブの Estimator を設定
*以下のデフォルト設定では、ハイパーパラメーター調整ジョブが完了するまでに10分程度かかります。*

この例では、 SageMaker Python SDK を使用して、ハイパーパラメーターの最適化を取り入れた学習を行います。先ほどはノートブックインスタンス上で学習を行うのローカルモードを扱いましたが、ここでは、`ml.m4.xlarge` を学習インスタンスとして個別に起動し、その上にPyTorchコンテナを起動し、学習する方法を説明します。

ハイパーパラメータ調整とは、ディープラーニングでよく用いられるパラメータの最適化の手法です。SageMaker は、学習率、バッチサイズ、エポック数などのパラメータの最適値を探索するためのインターフェースを持っています。探索したいハイパーパラメーターごとに、連続値として探索する場合はその範囲、またはリストの中から探索したい場合は探索可能な値のリストを指定します。ハイパーパラメーター調整ジョブは、異なるハイパーパラメーター設定で複数のトレーニングジョブを並列に起動し、定義済みの `objective metric` （目的関数） に基づいてそれらのトレーニングジョブの結果を評価します。以前の結果に基づいて次のハイパーパラメーター探索の設定を選択します。ハイパーパラメーター調整ジョブごとに、一度に並列で実行する学習インスタンス数、最大の学習ジョブ数、をそれぞれ割り当て、最大の学習ジョブ数が実行されると探索を終え、`objective metric` に対して最適なパフォーマンスを達成したハイパーパラメータの組み合わせを返します。

次に、以下の手順に従って、SageMaker Python SDK を使用してハイパーパラメーター調整ジョブをセットアップします。
* `estimator` を作成して PyTorch 学習ジョブをセットアップします。
* 調整するハイパーパラメーターの範囲を定義します。この例では、学習率とバッチサイズのハイパーパラメータ探索範囲を設定します。
* 最適化するチューニングジョブの `objective metric` を定義します。
* 上記の設定で `HyperparameterTuner` オブジェクトを設定します。

Spot Instanceを用いて実行する場合は、下記のコードを Estimator の train_instance_type の次の行に追加しましょう。

```python
                             max_run = 5000,
                             use_spot_instances = 'True',
                             max_wait = 10000,
```

In [ ]:
from sagemaker.pytorch import PyTorch

hpo_estimator = PyTorch(entry_point="mnist.py",
                    role=role,
                    framework_version='1.8.1',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'epochs': 4,
                        'backend': 'gloo'
                    })

`Estimator` を定義したら、調整するハイパーパラメーターとその探索範囲を指定します。ハイパーパラメーターの探索範囲の指定は3種類の方法があります。
- カテゴリーパラメーターは、探索したい値のリストを `CategoricalParameter（list）` で定義します。このリストの中から最適な値を探索します。 
- 連続パラメーターは、`ContinuousParameter（min、max）` で定義される最小値と最大値の間の連続空間で、任意の実数値から探索を行います。
- 整数パラメーターは、`IntegerParameter（min、max）` で定義された最小値と最大値の間の任意の整数値で、探索を行います。

* 可能であれば、値を最も制限の少ないタイプとして指定することをお勧めします。 たとえば、0.01〜0.2の連続値として学習率を調整すると、0.01、0.1、0.15、または0.2の値を持つカテゴリパラメーターとして調整するよりも良い結果が得られる可能性があります。 バッチサイズは一般的に2のべき乗であることが推奨されているため、ここではカテゴリパラメータとしてバッチサイズを指定しています。

In [ ]:
hyperparameter_ranges = {'lr': ContinuousParameter(0.001, 0.1),'batch-size': CategoricalParameter([32,64,128,256])}

次に objective metric とその定義を指定します。これには、トレーニングジョブの CloudWatch ログからそのメトリックを抽出するために必要な正規表現（Regex）が含まれます。 この特定のケースでは、スクリプトは平均損失値を出力し、それを objective metric として使用します。また、`objective_type` を `minimize` に設定します。これにより、ハイパーパラメーターチューニングは、最適なハイパーパラメーター設定を検索するときに客観的なメトリックを最小化しようとします。 デフォルトでは、 `objective_type` は `maximize` に設定されています。

In [ ]:
objective_metric_name = 'average test loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'average test loss',
                       'Regex': 'Test set: Average loss: ([0-9\\.]+)'}]

### 5.2.ハイパーパラメーター調整ジョブの tuner 設定する
次に、 `HyperparameterTuner` オブジェクトを作成します。
- 上記で作成した PyTorch 推定器
- ハイパーパラメーターの範囲
- Objective metric 名と定義
- 合計で実行するトレーニングジョブの数や並行して実行できるトレーニングジョブの数などのリソース構成の調整。

In [ ]:
tuner = HyperparameterTuner(hpo_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=4,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

### 5.3.ハイパーパラメーター調整ジョブを起動する
`.fit（）`を呼び出し、学習およびテストデータセットへの S3 パスを渡すことで、ハイパーパラメーターチューニングジョブを開始できます。

ハイパーパラメーターチューニングジョブが作成されたら、次のステップでチューニングジョブを記述してその進行状況を確認できます。SageMaker コンソールからジョブに移動して、ハイパーパラメーターチューニングジョブの進行状況を確認できます。

In [ ]:
tuner.fit({'training': inputs})
tuner.wait()

## 6.ホスティング
### 6.1.エンドポイントを作成する
トレーニング完了後、Tuner オブジェクトを使用して、`PyTorchPredictor` をビルドおよびデプロイします。前の手順では、Tuner が複数のトレーニングジョブを起動し、最適な obejctive metric を持つ結果のモデルが最適なモデルとして定義ました。これにより、SageMaker エンドポイントが作成されます。これにより、チューナーが探索した最適なモデルに基づいて推論を実行するためのホスティングを行います。

deploy 関数の引数により、エンドポイントに使用されるインスタンスの数とタイプを設定できます。これらは、トレーニングジョブで使用した値と同じである必要はありません。ここでは、モデルを単一の ```ml.m4.xlarge``` インスタンスにデプロイします。

In [ ]:
predictor_hpo = tuner.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### 6.2.評価
Estimator を使用して、手書きの数字を分類できるようになりました。Jupyer Notebook のみで利用可能です。 (Jupyter lab では利用できません）

下のセルを実行すると、空の画像ボックスが表示されます。 次に、その中に数字を描画すると、ピクセルデータがこのノートブックの `data` 変数にロードされ、`predictor` に渡すことができます。

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

### - HPOを用いた学習の結果

In [ ]:
import numpy as np
image = np.array([data], dtype=np.float32)

response_hpo = predictor_hpo.predict(image)
prediction_hpo = response_hpo.argmax(axis=1)[0]
print(prediction_hpo)

### - ローカルモードの学習の結果

In [ ]:
image = np.array([data], dtype=np.float32)

response = predictor.predict(image)
prediction  = response.argmax(axis=1)[0]
print(prediction)

## 7.リソースの削除
上記で作成したホスティングエンドポイントは、明示的に削除しないと立ち上がったままになり、課金が継続されます。不要な課金を防ぐために、このノートブックの実行が終了したらエンドポイントを削除しましょう。立ち上がっている同一リージョン内のエンドポイントは、SageMaker マネージメントコンソールの「推論」->「エンドポイント」から一覧を確認できます。

In [ ]:
# tuner で作ったエンドポイントの削除
predictor_hpo.delete_endpoint()